In [ ]:
!pip install -U bitsandbytes peft accelerate datasets sentencepiece wandb python-dotenv wtpsplit langchain
!pip install flash-attn --no-build-isolation
!pip install wtpsplit==2.1.1
!pip install syntok==1.4.4
!pip install omegaconf
!pip install wandb
!pip install --upgrade transformers trl
!pip install pandas numpy
!pip install gdown

In [ ]:
!git clone https://github.com/Reennon/gen-ai-nlp-lab-1

In [ ]:
%cd gen-ai-nlp-lab-1
!ls

In [ ]:
import os
import torch
import pandas as pd

from langchain_core.prompts import PromptTemplate
from transformers import AutoModelForCausalLM, AutoTokenizer, AutoConfig
from huggingface_hub import login
from transformers import PreTrainedTokenizerBase, BitsAndBytesConfig
from tqdm import tqdm
from torch.utils.data import Dataset
from datasets import Dataset
from src.prompts.prompts import (NERPrompt1, NERPrompt2, NERPrompt3, NERPrompt4)
from src.prompts.examples import (
    FIFTEEN_SHOT_EXAMPLES_DICT,
    ELEVEN_SHOT_EXAMPLES_DICT,
    FIVE_SHOT_EXAMPLES_DICT,
    THREE_SHOT_EXAMPLES_DICT,
    ZERO_SHOT_EXAMPLES_DICT
)
from omegaconf import OmegaConf
from google.colab import userdata
import gdown

In [ ]:
QUANTIZE_4BIT = False
# device   = "cuda:0"
device = "cuda:0"

In [ ]:
parameters = OmegaConf.load("./params/aya_23_8b.yml")

In [ ]:
login(userdata.get('hf_key'))

In [ ]:
checkpoint = "CohereForAI/aya-23-8b"
quantization_config = None
if QUANTIZE_4BIT:
  quantization_config = BitsAndBytesConfig(
      load_in_4bit=True,
      bnb_4bit_quant_type="nf4",
      bnb_4bit_use_double_quant=True,
      bnb_4bit_compute_dtype=torch.bfloat16,
  )
tokenizer = AutoTokenizer.from_pretrained(checkpoint)
seq_length = parameters.baseline.max_new_tokens
tokenizer.model_max_length = seq_length
max_seq_length = seq_length
config = AutoConfig.from_pretrained(checkpoint)
model = AutoModelForCausalLM.from_pretrained(
    checkpoint,
    config=config,
    quantization_config=quantization_config,
    torch_dtype="bfloat16",
    device_map="auto",  # Automatically map to GPUs
    attn_implementation="flash_attention_2",
)

In [ ]:
url = 'https://drive.google.com/uc?id=1kOOC_hgKBVK1HEew_DkfTdFLs-3cXkgT'
output = './data/silver_test.csv'
gdown.download(url, output, quiet=False)

In [ ]:
silver_test_df = pd.read_csv("./data/silver_test.csv")

In [ ]:
from src.prompts.prompts import BasePrompt

class NERPrompt1(BasePrompt):
    template: str = """Твоє завдання – виділити всі сутності у наданому тексті за наведеними категоріями та вивести їх у форматі JSON-списку:

Категорії сутностей ("label"):
- ART: артефакт (створений людиною предмет)
- DATE: дата (календарна дата, рік)
- DOC: документ (назви документів)
- JOB: посада (професійний титул, робоча позиція)
- LOG: місце (географічні об’єкти, назви країн, міст, річок тощо)
- MISC: різне (інші сутності, не підпадають під інші категорії)
- MON: гроші (сума, валюта)
- ORG: організація (установи, компанії, заклади)
- PCT: відсоток (число у відсотках)
- PERIOD: період (тривалість часу)
- PERS: особа (людські імена, прізвища)
- QUANT: кількість (числові значення)
- TIME: час (конкретний момент доби)

Формат відповіді: список об’єктів у JSON, кожен об’єкт має поля:
"label" – категорія сутності
"text" – фрагмент тексту сутності з оригінального тексту без змін

Не виводь дублікати знайдених сутностей.
Та не змінюй відмінків знайдених сутностей.
Виводь сутності з вхідного тексту, а не прикладів.
Перевір, чи справді знайдена сутність відповідає категорії сутностей.
Якщо знайдена сутність не відповідає її категорії, то її не слід включати у відповідь.

Нижче наведено приклади формату та стилю розпізнавання сутностей:
{examples}

ВХІДНИЙ ТЕКСТ:
{text}

ЗНАЙДЕНІ СУТНОСТІ:"""
    input_variables: list[str] = ["text", "examples"]

In [ ]:
example_template: str = """
ПРИКЛАД ТЕКСТУ:
{example_text}
ЗНАЙДЕНІ СУТНОСТІ З ТЕКСТУ:
{example_labels}
"""

def construct_prompt(
    prompt_template: PromptTemplate,
    few_shot_dict: dict[str, str],
    text: str
) -> str:
    examples = "".join([
            example_template.format(
                example_text=example["text"],
                example_labels=example["labels"]
            ) for example in few_shot_dict
        ])
    prompt = prompt_template.format(
        examples=examples,
        text=text,
    )

    return prompt

prompt = construct_prompt(
    prompt_template=NERPrompt1().prompt_template,
    few_shot_dict=ELEVEN_SHOT_EXAMPLES_DICT,
    text=""
)
print(prompt)

In [ ]:
tokenized_prompt = tokenizer.encode(prompt)
decoded_prompt = tokenizer.decode(tokenized_prompt)
prompt_len = len(tokenizer.tokenize(prompt))
print(decoded_prompt), prompt_len

In [ ]:
def get_message_format(prompts):
  messages = []

  for p in prompts:
    messages.append(
        [{"role": "user", "content": p}]
      )

  return messages

In [ ]:
model.eval()

out_labels = {}
max_seq_length = prompt_len + seq_length

batch_size_max = 4

for id in tqdm(silver_test_df.id.unique(), desc="Text progress"):
    inputs: list[str] = silver_test_df.loc[silver_test_df.loc[:, "id"] == id, "text"].to_list()
    inputs: list[str] = [construct_prompt(
        prompt_template=NERPrompt1().prompt_template,
        few_shot_dict=THREE_SHOT_EXAMPLES_DICT,
        text=input
    ) for input in inputs]
    print("inputs len")
    print([len(tokenizer.encode(input)) for input in inputs])
    print()
    inputs: list[dict[str, str]] = get_message_format(inputs)
    input_ids = tokenizer.apply_chat_template(
        inputs,
        tokenize=True,
        add_generation_prompt=True,
        padding=True,
        return_tensors="pt",
    )
    if len(input_ids) < batch_size_max:
      input_ids = input_ids.to(model.device)
      #input_ids = input_ids.to(model.device)
      prompt_padded_len = len(input_ids[0])
      # Generate corrections
      # Check if the model is wrapped in DataParallel
      gen_tokens = model.generate(
              input_ids,
              temperature=parameters.baseline.temperature,
              top_p=parameters.baseline.top_p,
              top_k=parameters.baseline.top_k,
              max_new_tokens=seq_length,
              do_sample=True,
      )

      gen_tokens = [
          gt[prompt_padded_len:] for gt in gen_tokens
      ]
      outputs: list[dict] = tokenizer.batch_decode(
          gen_tokens,
          skip_special_tokens=True
      )
    else:
      from torch.utils.data import DataLoader
      outputs: list[dict] = []
      dataloader = DataLoader(input_ids, batch_size=batch_size_max, shuffle=False)

      for batch in dataloader:
        batch = batch.to(device)
        prompt_padded_len = len(batch[0])
        gen_tokens = model.generate(
            batch,
            temperature=parameters.baseline.temperature,
            top_p=parameters.baseline.top_p,
            top_k=parameters.baseline.top_k,
            max_new_tokens=seq_length,
            do_sample=True,
        )
        gen_tokens = [gt[prompt_padded_len:] for gt in gen_tokens]
        outputs.extend(
            tokenizer.batch_decode(gen_tokens, skip_special_tokens=True)
        )


    print("output len")
    print(len(tokenizer.encode(str(outputs))))
    print("output")
    print(id, outputs)

    out_labels[id] = outputs

print(f"All texts extracted!")

In [ ]:
import ast
import json
import re

def clean_incomplete_entries(data: str) -> str:
    pattern = r"\{[^}]*\}"  # Matches everything enclosed in {}

    # Find all matches of complete entries
    complete_entries = re.findall(pattern, data)

    # Join back into a single cleaned string
    cleaned_data = ', '.join(complete_entries)

    return cleaned_data

def filter_unique_dicts(dict_list):
    seen = set()
    unique_dicts = []

    for d in dict_list:
        # Convert dictionary to a frozenset of its items (immutable and hashable)
        items = frozenset(d.items())
        if items not in seen:
            seen.add(items)
            unique_dicts.append(d)

    return unique_dicts

def join_split_dicts(entities):
  some_list = []
  for entity_list in entities:
    some_list.extend(entity_list)

  return some_list

def postprocess(entities):
  try:
    result_dict_list = []
    for e in entities:
      e = clean_incomplete_entries(e)
      e = ast.literal_eval(e)
      e = list(e)
      e = filter_unique_dicts(e)
      result_dict_list.extend(e)

    result_dict_list = filter_unique_dicts(result_dict_list)

  except Exception as e:
    print("skipping due to error, ", e)
    result_dict_list = []

  return result_dict_list

def filter_dicts_by_text(dict_list, original_text):
    return [d for d in dict_list if 'text' in d and d['text'] in original_text]

silver_test_predictions_df = pd.DataFrame.from_dict([out_labels]).T
silver_test_predictions_df.columns = ["entities"]
silver_test_predictions_df.index.name = "id"
silver_test_predictions_df = silver_test_predictions_df.reset_index()
silver_test_predictions_df.loc[:, "entities"] = \
  silver_test_predictions_df.loc[:, "entities"].apply(lambda e: postprocess(e))
silver_test_df_concatenated = (
    silver_test_df.groupby("id", as_index=False)["text"].apply(" ".join).reset_index()
)
silver_test_predictions_df = pd.merge(
    silver_test_predictions_df,
    silver_test_df_concatenated.loc[:, ["text", "id"]],
    how="left",
    on="id"
)
silver_test_predictions_df.loc[:, "entities"] = \
  silver_test_predictions_df.apply(lambda e: filter_dicts_by_text(e["entities"], e["text"]), axis=1)
silver_test_predictions_df.loc[:, "entities_dumps"] = \
  silver_test_predictions_df.loc[:, "entities"].apply(lambda e: json.dumps(e))
silver_test_predictions_df


In [ ]:
silver_test_predictions_df.to_csv(f"./data/submission_first_run.csv", index=False)

In [ ]:
submissions_df = pd.read_csv("./data/submission_first_run.csv")

not_scored_ids = submissions_df.loc[submissions_df.loc[:, "entities"] == "[]"]

In [ ]:
out_labels = {}

batch_size_max = 4

for id in tqdm(not_scored_ids.id.unique(), desc="Text progress"):
    inputs: list[str] = silver_test_df.loc[silver_test_df.loc[:, "id"] == id, "text"].to_list()
    inputs: list[str] = [construct_prompt(
        prompt_template=NERPrompt1().prompt_template,
        few_shot_dict=THREE_SHOT_EXAMPLES_DICT,
        text=input
    ) for input in inputs]
    print("inputs len")
    print([len(tokenizer.encode(input)) for input in inputs])
    print()
    inputs: list[dict[str, str]] = get_message_format(inputs)
    input_ids = tokenizer.apply_chat_template(
        inputs,
        tokenize=True,
        add_generation_prompt=True,
        padding=True,
        return_tensors="pt",
    )
    if len(input_ids) < batch_size_max:
      input_ids = input_ids.to(model.device)
      #input_ids = input_ids.to(model.device)
      prompt_padded_len = len(input_ids[0])
      # Generate corrections
      # Check if the model is wrapped in DataParallel
      gen_tokens = model.generate(
              input_ids,
              temperature=parameters.baseline.temperature,
              top_p=parameters.baseline.top_p,
              top_k=parameters.baseline.top_k,
              max_new_tokens=seq_length,
              do_sample=True,
      )

      gen_tokens = [
          gt[prompt_padded_len:] for gt in gen_tokens
      ]
      outputs: list[dict] = tokenizer.batch_decode(
          gen_tokens,
          skip_special_tokens=True
      )
    else:
      from torch.utils.data import DataLoader
      outputs: list[dict] = []
      dataloader = DataLoader(input_ids, batch_size=batch_size_max, shuffle=False)

      for batch in dataloader:
        batch = batch.to(device)
        prompt_padded_len = len(batch[0])
        gen_tokens = model.generate(
            batch,
            temperature=parameters.baseline.temperature,
            top_p=parameters.baseline.top_p,
            top_k=parameters.baseline.top_k,
            max_new_tokens=seq_length,
            do_sample=True,
        )
        gen_tokens = [gt[prompt_padded_len:] for gt in gen_tokens]
        outputs.extend(
            tokenizer.batch_decode(gen_tokens, skip_special_tokens=True)
        )

    print("output len")
    print(len(tokenizer.encode(str(outputs))))
    print("output")
    print(id, outputs)

    out_labels[id] = outputs

print(f"All non scored texts has been scored!")

In [ ]:
silver_test_predictions_df = pd.DataFrame.from_dict([out_labels]).T
silver_test_predictions_df.columns = ["entities"]
silver_test_predictions_df.index.name = "id"
silver_test_predictions_df = silver_test_predictions_df.reset_index()
silver_test_predictions_df.loc[:, "entities"] = \
  silver_test_predictions_df.loc[:, "entities"].apply(lambda e: postprocess(e))
silver_test_df_concatenated = (
    silver_test_df.groupby("id", as_index=False)["text"].apply(" ".join).reset_index()
)
silver_test_predictions_df = pd.merge(
    silver_test_predictions_df,
    silver_test_df_concatenated.loc[:, ["text", "id"]],
    how="left",
    on="id"
)
silver_test_predictions_df.loc[:, "entities"] = \
  silver_test_predictions_df.apply(lambda e: filter_dicts_by_text(e["entities"], e["text"]), axis=1)
silver_test_predictions_df.loc[:, "entities_dumps"] = \
  silver_test_predictions_df.loc[:, "entities"].apply(lambda e: json.dumps(e))
silver_test_predictions_df

In [ ]:
silver_test_predictions_df.to_csv(f"./data/submission_second_run.csv", index=False)

In [ ]:
submissions_second_run_df = pd.read_csv(f"./data/submission_second_run.csv")
submissions_second_run_df = submissions_second_run_df.reset_index(drop=True)
submissions_second_run_df = submissions_second_run_df.drop_duplicates(subset="id")
submissions_second_run_df

In [ ]:
submissions_df = pd.read_csv("../data/submission_merged.csv")
print(len(submissions_df.loc[submissions_df.loc[:, 'entities'] == "[]"]))
submissions_df = pd.concat([
    submissions_df.loc[~submissions_df.loc[:, "id"].isin(submissions_second_run_df.id)],
    submissions_second_run_df,
])
print(len(submissions_df.loc[submissions_df.loc[:, 'entities'] == "[]"]))
submissions_df


In [ ]:
submissions_df = submissions_df.loc[:, ["id", "entities_dumps"]]
submissions_df.columns = ["id", "entities"]
submissions_df.to_csv("../data/submission.csv", index=False)